# Оценка баланса реактивной мощности

## Проверка по условию выполнения БРМ в сети и определение приведенных к шинам высшего напряжения нагрузок подстанции 

In [27]:
from IPython.display import display, Math
import numpy as np
%store -r Q_nb_i
%store -r S_nb_i
%store -r Lij

In [2]:
#                  K1    1-3    3-2   3-4   4-5
P_1c = np.array([125.3, 119.7, 55.3, 49.7, 37.5])
L_lep = np.array([Lij[0, 1], Lij[1, 3], Lij[3, 2], Lij[3, 4], Lij[4, 5]])
mask_p1c = [False, False, True, True, True]

In [3]:
k_odn_q = 0.98
mask_ps = [False, True, False, True, True]
Q_treb_1 = np.around(k_odn_q * np.sum(Q_nb_i), 1)
print(f'Q_treb_1 = {Q_treb_1}')

Q_treb_2 = 0.08 * np.sum(np.where(mask_ps, S_nb_i * 2, S_nb_i))
print(f'Q_treb_2 = {Q_treb_2}')

q_c0_220 = 0.14
q_c0_110 = 0.036
i = 0
Q_c = np.zeros_like(L_lep)
for i in range(len(L_lep)):
    if mask_p1c[i]:
        Q_c[i] = np.around(2 * q_c0_110 * L_lep[i], 2)
    else:
        Q_c[i] = np.around(2 * q_c0_220 * L_lep[i], 2)
print(f'Q_c = {Q_c}')

Q_treb_1 = 93.5
Q_treb_2 = 33.496
Q_c = [11.7  13.78  3.01  1.86  1.86]


In [12]:
np.sum(Q_c)

32.209999999999994

In [13]:
np.sum(delta_Q_l)

40.790000000000006

In [4]:
P_nat_220 = 130
P_nat_110 = 30
i = 0
delta_Q_l = np.zeros_like(Q_c)
for i in range(len(Q_c)):
    if mask_p1c[i] and P_1c[i] > P_nat_110:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_110)**2 * Q_c[i], 2)
    elif mask_p1c[i] and P_1c[i] <= P_nat_110:
        delta_Q_l[i] = Q_c[i]
    else:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_220)**2 * Q_c[i], 2)
print(f'delta_Q_l = {delta_Q_l}')

Q_treb_3 = np.around(np.sum(delta_Q_l - Q_c), 1)

Q_treb_sum = np.around(Q_treb_1 + Q_treb_2 + Q_treb_3, 1)
print(f'Q_treb_3 = {Q_treb_3} \nQ_treb_sum = {Q_treb_sum}')

delta_Q_l = [10.87 11.68 10.23  5.1   2.91]
Q_treb_3 = 8.6 
Q_treb_sum = 135.6


## Расстановка КУ

In [17]:
%store -r P_nb
%store -r tg_nb_i
q_rasp = 87
Q_dop_sum = Q_treb_sum - q_rasp
tg_phi_b = np.around((np.sum(Q_nb_i) - Q_dop_sum) / np.sum(P_nb), 3) # высчитываем для всех подстанций
print(tg_phi_b)
tg_phi_b = np.around((np.sum(np.where(tg_nb_i < tg_phi_b, 0, Q_nb_i)) - Q_dop_sum) / np.sum(np.where(tg_nb_i < tg_phi_b, 0, P_nb)), 3) # уточняем, если на какой-то подстанции не нужно
print(tg_phi_b)

0.191
0.191


In [15]:
Q_comp_dop

array([14. , 14. ,  5.6,  8. ,  6.9])

In [6]:
Q_comp_dop = np.around(P_nb * (tg_nb_i - tg_phi_b), 1)
print("Q_comp_dop = ", Q_comp_dop)
N_bsk = np.around(Q_comp_dop / 1.2, 1)
N_bsk_treb = np.around(Q_dop_sum / 1.2, 1)
print("N_bsk до округления = ", N_bsk)
N_bsk_sum = 42
print("N_bsk_treb = ", N_bsk_treb)
N_bsk = np.array([10, 12, 6, 8, 6]) # уточняем количество БСК вручную
print("N_bsk_sum = ", N_bsk_sum)
print("N_bsk = ", N_bsk, "\nN_bsk = N_bsk_sum? ---", np.sum(N_bsk) == N_bsk_sum)

Q_comp_dop =  [14.  14.   5.6  8.   6.9]
N_bsk до округления =  [11.7 11.7  4.7  6.7  5.8]
N_bsk_treb =  40.5
N_bsk_sum =  42
N_bsk =  [10 12  6  8  6] 
N_bsk = N_bsk_sum? --- True


In [16]:
Q_bsk_dop = N_bsk * 1.2
Q2_nb = Q_nb_i - Q_bsk_dop # здесь Q2_nb это Q'_нбi и далее двоечка будет означать апостроф
S2_nb = np.around(np.sqrt(P_nb**2 + Q2_nb**2), 1)
Q_priv = np.around(Q2_nb + 0.08 * S2_nb, 1)
S_priv = np.around(np.sqrt(P_nb**2 + Q_priv**2), 1)

Q_priv[3-1] = np.around(Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + Q_priv[1] + 0.08 * 
                        np.sqrt((P_nb[2] + P_nb[3] + P_nb[4] + P_nb[1])**2 + (Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + Q_priv[1])**2), 1)
S_priv[3-1] = np.around(np.sqrt((P_nb[2] + P_nb[3] + P_nb[4] + P_nb[1])**2 + Q_priv[2]), 1)

print("Q_bsk_dop = ", Q_bsk_dop, "\nQ''_nb = ", Q2_nb, "\nS''_nb = ", S2_nb, "\nQ_priv = ", Q_priv, "\nS_priv = ", S_priv)

Q_bsk_dop =  [12.  14.4  7.2  9.6  7.2] 
Q''_nb =  [15.4 13.   4.1  6.1  6.4] 
S''_nb =  [71.7 71.2 30.3 40.5 35.6] 
Q_priv =  [21.1 18.7 55.7  9.3  9.2] 
S_priv =  [ 73.1  72.5 175.2  41.1  36.2]


In [30]:
display(Math(f'''Q_{{прив3}} = Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + Q_{{прив2}} + 0,08 \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}} + P_{{нб2}})^2 + (Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + Q_{{прив2}})^2}} = 
             {Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + {Q_priv[1]} + 0,08 \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]} + {P_nb[1]})^2 + ({Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + {Q_priv[1]})^2}} = {Q_priv[3-1]}'''))

<IPython.core.display.Math object>

In [31]:
display(Math(f'''S_{{прив3}} = \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}} + P_{{нб2}})^2 + (Q_{{прив3}})^2}} = 
                \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]} + {P_nb[1]})^2 + ({Q_priv[2]})^2}} = {S_priv[3-1]}'''))

<IPython.core.display.Math object>